In [1]:
import os
import pickle
from astropy.time import Time
from lsst_efd_client import EfdClient
#from lsst.resources import ResourcePath

from rubin_sim.scheduler.schedulers import CoreScheduler
from rubin_sim.scheduler.features import Conditions
from rubin_sim.site_models import Almanac

#from lsst.rsp import get_access_token

In [2]:
efd_client = EfdClient("usdf_efd")

topics = await efd_client.get_topics()
[t for t in topics if "Scheduler" in t]
#topics

['lsst.sal.Scheduler.ackcmd',
 'lsst.sal.Scheduler.command_addBlock',
 'lsst.sal.Scheduler.command_disable',
 'lsst.sal.Scheduler.command_enable',
 'lsst.sal.Scheduler.command_exitControl',
 'lsst.sal.Scheduler.command_resume',
 'lsst.sal.Scheduler.command_setLogLevel',
 'lsst.sal.Scheduler.command_standby',
 'lsst.sal.Scheduler.command_start',
 'lsst.sal.Scheduler.command_stop',
 'lsst.sal.Scheduler.logevent_authList',
 'lsst.sal.Scheduler.logevent_blockInventory',
 'lsst.sal.Scheduler.logevent_blockStatus',
 'lsst.sal.Scheduler.logevent_configurationApplied',
 'lsst.sal.Scheduler.logevent_configurationsAvailable',
 'lsst.sal.Scheduler.logevent_dependenciesVersions',
 'lsst.sal.Scheduler.logevent_detailedState',
 'lsst.sal.Scheduler.logevent_domeConfig',
 'lsst.sal.Scheduler.logevent_errorCode',
 'lsst.sal.Scheduler.logevent_generalInfo',
 'lsst.sal.Scheduler.logevent_heartbeat',
 'lsst.sal.Scheduler.logevent_largeFileObjectAvailable',
 'lsst.sal.Scheduler.logevent_logLevel',
 'lsst.s

In [3]:
await efd_client.get_fields('lsst.sal.Scheduler.logevent_surveyTopology')

['generalPropos',
 'numGeneralProps',
 'numSeqProps',
 'private_efdStamp',
 'private_identity',
 'private_kafkaStamp',
 'private_origin',
 'private_rcvStamp',
 'private_revCode',
 'private_seqNum',
 'private_sndStamp',
 'salIndex',
 'sequencePropos']

In [4]:
good_locs = {"USDF" : "usdf", "SUMMIT": "summit"}

loc = "usdf"
efd_client = EfdClient(f"{loc}_efd")

# Look at EFD events in a particular night
night = Time('2023-10-11', format='isot')
almanac = Almanac()
sunset = Time(almanac.get_sunset_info(night.mjd)[1], format='mjd', scale='utc')
sunrise = Time(almanac.get_sunset_info(night.mjd)[-1], format='mjd', scale='utc')

# Get all scheduler snapshots for a given night
topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
fields = ["url"]

# or if we wanted all exposures taken in that night 
#topic = "lsst.sal.Scheduler.logevent_observation"
#fields = ["mjd", "ra", "decl", "filter", "rotSkyPos", "exptime", "nexp", "targetId", "additionalInfo"]

dd = await efd_client.select_time_series(topic, fields, sunset, sunrise, index=2)
dd

,url
2023-10-10 23:57:26.220941+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2023-10-11 00:09:08.652330+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2023-10-11 00:15:16.909856+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2023-10-11 00:30:49.973907+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2023-10-11 00:35:23.006927+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
...,...
2023-10-11 08:44:29.629887+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2023-10-11 08:48:58.712387+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2023-10-11 08:58:01.292416+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2023-10-11 08:58:11.318659+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...


In [6]:
import pandas
pandas.__version__

'2.1.0'

In [5]:
url = dd.url[0]
from urllib.parse import urlparse
bucket = "s3://rubin:"
uri = ResourcePath(bucket + urlparse(url).path.lstrip('/'))

In [6]:
token_path = '/Users/lynnej/lj_rsp_token'
os.environ['ACCESS_TOKEN'] = token_path
get_access_token()

os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu"
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"

In [7]:
result = uri.read()

In [8]:
sched, conditions = pickle.loads(result)
assert isinstance(sched, CoreScheduler)
assert isinstance(conditions, Conditions)